In [141]:
import os
import glob

import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

from keplergl import KeplerGl

# Data

### CSV

### gouv data

In [142]:
#gouv data
covid_gouv = 'data/donnees-hospitalieres-covid19-2020-04-13-19h00.csv'
df_gouv    = pd.read_csv(covid_gouv, delimiter = ';')
df_gouv.head()

,dep,sexe,jour,hosp,rea,rad,dc
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6


In [143]:
#day data
df_jour = df_gouv['jour'].drop_duplicates()
max = df_jour.max()

In [144]:
#gouv preprocessing
df_gouv['code'] = df_gouv['dep']

temp_df = dict()
for i in df_jour :
    temp_df[i] = df_gouv.loc[ df_gouv['jour'] == i]
dict_df = dict()
for key, df in temp_df.items() :
    dict_df[key] = df.groupby('code').max().drop(['sexe','dep'], axis=1)
dict_df[max].head()

,jour,hosp,rea,rad,dc
code,,,,,
01,2020-04-13,120,30,139,44
02,2020-04-13,286,44,288,147
03,2020-04-13,58,20,82,13
04,2020-04-13,33,4,63,5
05,2020-04-13,49,13,70,3


### demographic data

In [145]:
# Source : https://www.regions-et-departements.fr/departements-francais
demographie = 'data/demographie (3).csv'
df_de       = pd.read_csv(demographie, delimiter = ',')
df_de       = df_de.set_index('code')
df_de.head()

,nom,Superficie,Population,Densite
code,,,,
01,Ain,5762,631877,109.7
02,Aisne,7369,538659,73.1
03,Allier,7340,341613,46.5
04,Alpes-de-Haute-Provence,6925,161799,23.4
05,Hautes-Alpes,5549,140916,25.4


In [146]:
#Source : https://www.insee.fr/fr/statistiques/2012692#graphique-TCRD_021_tab1_departements
demographie_1  = 'data/TCRD_021.csv'
df_de1         = pd.read_csv(demographie_1, delimiter = ',')
df_de1['code'] = df_de1['Unnamed: 0']
df_de1         = df_de1.set_index('code').drop(['Unnamed: 0','Unnamed: 1'], axis =1 )
df_de1         = df_de1.dropna()
df_de1

,Ensemble,Part des femmes (en %),Part des hommes (en %),Part des 0 à 24 ans (en %),Part des 25 à 59 ans (en %),Part des 60 ans ou plus (en %),dont part des 75 ans ou plus (en %)
code,,,,,,,
01,656955.0,50.8,49.2,30.2,45.6,24.2,8.3
02,526050.0,51.1,48.9,29.5,42.6,27.9,9.3
03,331315.0,52.2,47.8,24.3,40.2,35.6,13.8
04,165197.0,51.2,48.8,24.5,41.2,34.3,12.7
05,141756.0,51.2,48.8,25.2,42.4,32.4,11.9
...,...,...,...,...,...,...,...
971,376879.0,54.6,45.4,28.8,43.4,27.9,9.1
972,358749.0,54.2,45.8,25.9,43.3,30.8,10.6
973,290691.0,51.4,48.6,48.6,41.7,9.7,2.1


### Geospatial data

In [162]:
geospatial_data  = 'data/departements-avec-outre-mer.geojson'
f = gpd.read_file(geospatial_data)
f = f.set_index('code')
f = f.sort_index(axis=0)
f.head()

,nom,geometry
code,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ..."
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ..."
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ..."
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ..."


# DataFrame Coronavirus

In [148]:
dict_coronavirus = dict()
for i in df_jour :
    dict_coronavirus[i] = pd.concat([f,dict_df[i]], axis = 1)
    dict_coronavirus[i]['dc/densite']    = dict_coronavirus[i]['dc'] / df_de['Densite']
    dict_coronavirus[i]['people older than 75 year (%)'] = df_de1['dont part des 75 ans ou plus (en %)']
    dict_coronavirus[i]['dc/population'] = dict_coronavirus[i]['dc'] / df_de['Population']
    dict_coronavirus[i]['dc/cumul hospitalise'] = dict_coronavirus[i]['dc'] / (dict_coronavirus[i]['hosp']+dict_coronavirus[i]['rad'])
    dict_coronavirus[i]['cumul hopitalise / population']     = (dict_coronavirus[i]['hosp']+dict_coronavirus[i]['rad']) / df_de['Population']
dict_coronavirus[max].head()

,nom,geometry,jour,hosp,rea,rad,dc,dc/densite,people older than 75 year (%),dc/population,dc/cumul hospitalise,cumul hopitalise / population
code,,,,,,,,,,,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2020-04-13,120,30,139,44,0.401094,8.3,0.000070,0.169884,0.000410
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2020-04-13,286,44,288,147,2.010944,9.3,0.000273,0.256098,0.001066
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2020-04-13,58,20,82,13,0.279570,13.8,0.000038,0.092857,0.000410
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2020-04-13,33,4,63,5,0.213675,12.7,0.000031,0.052083,0.000593
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2020-04-13,49,13,70,3,0.118110,11.9,0.000021,0.025210,0.000844


# Date

In [149]:
print(df_jour.min())
print(df_jour.max())

2020-03-18
2020-04-13


In [150]:
jour = '2020-04-13'

## Analyse

In [151]:
daily_coronavirus = dict_coronavirus[jour]
daily_coronavirus.head()

,nom,geometry,jour,hosp,rea,rad,dc,dc/densite,people older than 75 year (%),dc/population,dc/cumul hospitalise,cumul hopitalise / population
code,,,,,,,,,,,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2020-04-13,120,30,139,44,0.401094,8.3,0.000070,0.169884,0.000410
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2020-04-13,286,44,288,147,2.010944,9.3,0.000273,0.256098,0.001066
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2020-04-13,58,20,82,13,0.279570,13.8,0.000038,0.092857,0.000410
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2020-04-13,33,4,63,5,0.213675,12.7,0.000031,0.052083,0.000593
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2020-04-13,49,13,70,3,0.118110,11.9,0.000021,0.025210,0.000844


In [152]:
dc   = daily_coronavirus['dc'].sum()    #Nombre cumulé de décés
hosp = daily_coronavirus['hosp'].sum()  #Nombre de personne actuellement hospitalisé
rea  = daily_coronavirus['rea'].sum()   #Nombre de personne actuellement en réanimation
rad  = daily_coronavirus['rad'].sum()   #Nombre cumulé de personnes sorties de l'hospital 
print('==================================================================================')
print('')
print('chiffre du {}'.format(jour))
print('')
print('==================================================================================')

print('hospitalisation = {}'.format(hosp))
print('reanimation = {}'.format(rea))
print('sortie hospital cumulé= {}'.format(rad))
print('décés cumulé= {}'.format(dc))


chiffre du 2020-04-13

hospitalisation = 31952
reanimation = 6690
sortie hospital cumulé= 27717
décés cumulé= 9588


##  décés/population

In [153]:
print('')
print('=============================================================================')
print('')

#Haut-Rhin dc/population = 0.00075
maxdcpopulation = daily_coronavirus['dc/population'].max()
print(daily_coronavirus.loc[daily_coronavirus['dc/population'] == maxdcpopulation ]['nom'][0])
print(str(100 * daily_coronavirus.loc[daily_coronavirus['dc/population'] == maxdcpopulation ]['dc/population'][0]) + ' % ')


print('')
print('=============================================================================')
print('')

#Lozère, cantal ,Guyane, La Reunion car 0 dc
mindcdensite = daily_coronavirus['dc/population'].min()
print(daily_coronavirus.loc[daily_coronavirus['dc/population'] == mindcdensite ]['nom'][0])
print(str( 100 * daily_coronavirus.loc[daily_coronavirus['dc/population'] == mindcdensite ]['dc/population'][0]) + ' % ')

print('')
print('=============================================================================')
print('')

#Paris dc/population = 0.000393
departement = 'Paris'
print(departement)
print(str(100 * daily_coronavirus.loc[daily_coronavirus['nom'] == departement ]['dc/population'][0]) + ' % ')

print('')
print('=============================================================================')
print('')

#Bouche du rhone dc/population = 8.578702404317715e-05
departement = 'Bouches-du-Rhône'
print(departement)
print(str(100 * daily_coronavirus.loc[daily_coronavirus['nom'] == departement ]['dc/population'][0]) + ' % ')



Territoire de Belfort
0.07959413910287023 % 


Cantal
0.0 % 


Paris
0.04346273353854632 % 


Bouches-du-Rhône
0.009024993280842913 % 


## décés/densité de population

In [154]:
#Moselle max dc alors que densite faible
maxdcdensite = daily_coronavirus['dc/densite'].max()
print(daily_coronavirus.loc[daily_coronavirus['dc/densite'] == maxdcdensite ]['nom'])
print(maxdcdensite)

print('')
print('=============================================================================')
print('')

#Lozère, cantal ,Guyane, La Reunion car 0 dc
mindcdensite = daily_coronavirus['dc/densite'].min()
print(daily_coronavirus.loc[daily_coronavirus['dc/densite'] == mindcdensite ]['nom'])
print(mindcdensite)

print('')
print('=============================================================================')
print('')

#Paris dc/densite = 0.04
departement = 'Paris'
print(departement)
print(daily_coronavirus.loc[daily_coronavirus['nom'] == departement ]['dc/densite'])

code
57    Moselle
Name: nom, dtype: object
2.9583333333333335


code
15         Cantal
48         Lozère
973        Guyane
974    La Réunion
Name: nom, dtype: object
0.0


Paris
code
75    0.04581
Name: dc/densite, dtype: float64


## décés / nombre de cas

In [155]:
print('')
print('=============================================================================')
print('')

#Cher dc/cumul hospitalisé = 33.01 % 
maxdcpopulation = daily_coronavirus['dc/cumul hospitalise'].max()
print(daily_coronavirus.loc[daily_coronavirus['dc/cumul hospitalise'] == maxdcpopulation ]['nom'][0])
print(str(100 * daily_coronavirus.loc[daily_coronavirus['dc/cumul hospitalise'] == maxdcpopulation ]['dc/cumul hospitalise'][0]) + ' % ')


print('')
print('=============================================================================')
print('')

#Lozère, cantal ,Guyane, La Reunion car 0 dc
mindcdensite = daily_coronavirus['dc/cumul hospitalise'].min()
print(daily_coronavirus.loc[daily_coronavirus['dc/cumul hospitalise'] == mindcdensite ]['nom'][0])
print(str( 100 * daily_coronavirus.loc[daily_coronavirus['dc/cumul hospitalise'] == mindcdensite ]['dc/cumul hospitalise'][0]) + ' % ')

print('')
print('=============================================================================')
print('')

#Paris dc/cumul hospitalisé = 17.26 % 
departement = 'Paris'
print(departement)
print(str(100 * daily_coronavirus.loc[daily_coronavirus['nom'] == departement ]['dc/cumul hospitalise'][0]) + ' % ')

print('')
print('=============================================================================')
print('')

#Bouche du rhone dc/cumul hospitalisé = 7.59 %
departement = 'Bouches-du-Rhône'
print(departement)
print(str(100 * daily_coronavirus.loc[daily_coronavirus['nom'] == departement ]['dc/cumul hospitalise'][0]) + ' % ')



Cher
33.0188679245283 % 


Cantal
0.0 % 


Paris
17.269944174320187 % 


Bouches-du-Rhône
7.592824363788068 % 


### calcul de risque

#### demographie

In [156]:
# Part des 75 ans ou plus (en %)

cat_0 = 0
#Risque = 1
cat_1 = 9.000000 
#Risque = 2
cat_2 = 10.600000
#Risque = 3
cat_3 = 12.500000
#Risque = 4  

df_temp = df_de1.drop(['Ensemble','Part des femmes (en %)', 'Part des hommes (en %)','Part des 0 à 24 ans (en %)', 'Part des 25 à 59 ans (en %)', 'Part des 60 ans ou plus (en %)'], axis = 1)
x = df_temp['dont part des 75 ans ou plus (en %)']
df_temp['age_risque'] = df_temp['dont part des 75 ans ou plus (en %)'].apply(lambda x : 1 if x < cat_1 else (2 if x > cat_1 and x < cat_2 else (3 if x > cat_2 and x < cat_3 else 4)))
df_temp = df_temp.drop(['dont part des 75 ans ou plus (en %)'], axis = 1)
df_temp

,age_risque
code,
01,1
02,2
03,4
04,4
05,3
...,...
971,2
972,4
973,1


#### contamination

In [157]:
# Cumul hospitalisé par rapport à la population

cat_0 = 0
#Risque = 1
cat_1 = 0.000284
#Risque = 2
cat_2 = 0.000494
#Risque =  3
cat_3 = 0.000898
#Risque = 3

x1 = dict_coronavirus[max]['cumul hopitalise / population']
df_temp['contamination_risque'] = dict_coronavirus[max]['cumul hopitalise / population'].apply(lambda x : 4 if x < cat_1 else (3 if x > cat_1 and x < cat_2 else (2 if x > cat_2 and x < cat_3 else 1)))

In [158]:
 df_temp['score'] = df_temp['age_risque'] * df_temp['contamination_risque']

In [159]:
df_risque = pd.concat([f,df_temp], axis = 1)
df_risque

,nom,geometry,age_risque,contamination_risque,score
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",1,3,3
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2,1,2
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",4,3,12
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",4,2,8
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",3,2,6
...,...,...,...,...,...
971,Guadeloupe,"MULTIPOLYGON (((-61.79038 16.16682, -61.79164 ...",2,4,8
972,Martinique,"MULTIPOLYGON (((-60.86733 14.38867, -60.86802 ...",4,4,16
973,Guyane,"MULTIPOLYGON (((-53.87049 5.74494, -53.86464 5...",1,4,4
974,La Réunion,"MULTIPOLYGON (((55.56891 -21.37670, 55.56843 -...",1,4,4


### Visualisation

### Map 2D

#### Décés

In [121]:
%run config_2D.py
config_2D = config

In [122]:
map_coronavirus_2D = KeplerGl(height=500, config = config)
map_coronavirus_2D.add_data(dict_coronavirus['2020-04-13'], name ='coronavirus')
map_coronavirus_2D

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'eh1a0e', 'type': '…

In [123]:
config_2D = map_coronavirus_2D.config

In [124]:
# Save map_1 config to a file
with open('config_2D.py', 'w') as f:
    f.write('config = {}'.format(config_2D))

In [125]:
# # this will save current map
# map_coronavirus_2D.save_to_html(file_name='map_coronavirus_2D.html')

#### Risques

In [126]:
%run config_risque.py
config_risque = config

In [127]:
map_risque = KeplerGl(height=500, config = config_risque)
map_risque.add_data(df_risque, name = 'risque')
map_risque

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 's2l46au', 'type': …

In [136]:
config_risque = map_risque.config

In [137]:
# Save map_1 config to a file
with open('config_risque.py', 'w') as f:
    f.write('config = {}'.format(config_risque))

In [138]:
# this will save current map
map_risque.save_to_html(file_name='map_risque.html')

Map saved to map_risque.html!


### Map 3D

In [131]:
%run config_3D.py
config_3D = config

In [132]:
map_coronavirus_3D = KeplerGl(height=500, config = config_3D)
map_coronavirus_3D.add_data(daily_coronavirus, name ='coronavirus')
map_coronavirus_3D

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'eh1a0e', 'type': '…

In [133]:
config_3D = map_coronavirus_3D.config

In [134]:
# Save map_1 config to a file
with open('config_3D.py', 'w') as f:
    f.write('config = {}'.format(config_3D))

In [135]:
# this will save current map
map_coronavirus_3D.save_to_html(file_name='map_coronavirus_3D.html')

Map saved to map_coronavirus_3D.html!
